# Dynamical Decoupling (DD) Simulation

Ideal system:  **$e^{-iZt}$** \
Noisy system: **$e^{-i(Z+\lambda{X})t}$**
#### Want to prove: 
**$e^{-i(Z+\lambda{X})t}Ze^{-i(Z+\lambda{X})t}Z \approx e^{-2iZt}$** 

### Libraries and Pauli Matrices 

In [1]:
import numpy as np
import scipy
import math
from scipy.linalg import expm
import matplotlib.pyplot as plt

In [2]:
X, Z, I = np.matrix([[0, 1],[1, 0]]), np.matrix([[1, 0],[0, -1]]), np.matrix([[1, 0],[0, 1]])

### Simulation code

In [3]:
# creating the variables
time = []
epsilon = []

# set the value of lambda
lambda_value = 0.1

# simulating the equation for every 0.1s for the first 10s.
for t in np.linspace(0,1,0.01):
    A = expm(-1j*(Z+lambda_value*X)*t)*Z*expm(-1j*(Z+lambda_value*X)*t)*Z
    B = expm(-2j*Z*t)

    # calculate the difference between the ideal and noisy system
    difference = A - B
    # get the maximum eigen value of the difference matrix
    eigenvalues,_ = np.linalg.eig(difference)
    max_eigenvalue = eigenvalues.max()
    # add the maximum eigen value to the array epsilon
    epsilon.append(max_eigenvalue)
    #print(epsilon)
    #print(A)
    #print(B)
    #print(difference)
    time.append(t)


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
# simulating expression for the error matrix obtained from hand-calculation

error_matrix = []


for t in range(0,100,1):
    E = I*math.sin(t*math.sqrt(1+lambda_value**2))**2 + (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(I*(1-lambda_value**2)+2*lambda_value*X*Z)/math.sqrt(1+lambda_value**2)
    
    # get maximum eigen value
    ev,_ = np.linalg.eig(E)
    max_ev = ev.max()

    # add maximum eigen value to the array error_matrix
    error_matrix.append(max_ev)

    #print(E)

In [ ]:
other_A = []
for t in range(0,100,1):
    #A1 = (I*math.cos(t*math.sqrt(1+lambda_value**2))-1j*math.sin(t*math.sqrt(1+lambda_value**2))*(Z+lambda_value*X)/math.sqrt(1+lambda_value**2))
    #A2 = (I*math.cos(t*math.sqrt(1+lambda_value**2))-1j*math.sin(t*math.sqrt(1+lambda_value**2))*(Z-lambda_value*X)/math.sqrt(1+lambda_value**2))

    #works ->
    #A1 = I*math.cos(t*math.sqrt(1+lambda_value**2))**2 - 1j*math.cos(t*math.sqrt(1+lambda_value**2))*math.sin(t*math.sqrt(1+lambda_value**2))*(Z-lambda_value**2)/math.sqrt(1+lambda_value**2)
    #A2 = -1j*math.sin(t*math.sqrt(1+lambda_value**2))*math.cos(t*math.sqrt(1+lambda_value**2))*(Z+lambda_value**2)/math.sqrt(1+lambda_value**2)
    #A3 = (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(Z+lambda_value*X)*(Z-lambda_value*X)/math.sqrt(1+lambda_value**2) 
   
    #A1 = I*math.cos(t*math.sqrt(1+lambda_value**2))**2 -2j*math.cos(t*math.sqrt(1+lambda_value**2))*math.sin(t*math.sqrt(1+lambda_value**2))*Z
    #A2 = (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(I+2*lambda_value*X*Z-I*lambda_value**2)/math.sqrt(1+lambda_value**2)

    #A1 = I*math.cos(t*math.sqrt(1+lambda_value**2))**2 -1j*math.sin(2*t*math.sqrt(1+lambda_value**2))*Z/math.sqrt(1+lambda_value**2)
    #A2 = (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(I+2*lambda_value*X*Z-I*lambda_value**2)/math.sqrt(1+lambda_value**2)

    #A1 = I/2 + I*math.cos(2*t*math.sqrt(1+lambda_value**2))/2 -1j*math.sin(2*t*math.sqrt(1+lambda_value**2))*Z/math.sqrt(1+lambda_value**2)
    #A2 = (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(I*(1-lambda_value**2)+2*lambda_value*X*Z)/math.sqrt(1+lambda_value**2)

    #A1 = I*math.cos(2*t*math.sqrt(1+lambda_value**2))-1j*math.sin(2*t*math.sqrt(1+lambda_value**2))*Z/math.sqrt(1+lambda_value**2)
    #A2 = I/2 - I*math.cos(2*t*math.sqrt(1+lambda_value**2))/2 + (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(I*(1-lambda_value**2)+2*lambda_value*X*Z)/math.sqrt(1+lambda_value**2)

    A1 = I*math.cos(2*t*math.sqrt(1+lambda_value**2))-1j*math.sin(2*t*math.sqrt(1+lambda_value**2))*Z/math.sqrt(1+lambda_value**2)
    A2 = I*math.sin(t*math.sqrt(1+lambda_value**2))**2 + (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(I*(1-lambda_value**2)+2*lambda_value*X*Z)/math.sqrt(1+lambda_value**2)

    #A1 = expm(-2j*Z*t)
    #A2 = I*math.sin(t*math.sqrt(1+lambda_value**2))**2 + (-1*math.sin(t*math.sqrt(1+lambda_value**2))**2)*(I*(1-lambda_value**2)+2*lambda_value*X*Z)/math.sqrt(1+lambda_value**2)
    
    #A1 = I - 1j*(Z+lambda_value*X)*t + 0.5*(1j*(Z+lambda_value*X)*t)**2 
    #A2 = I - 1j*(Z-lambda_value*X)*t + 0.5*(1j*(Z-lambda_value*X)*t)**2 

    
    B = expm(-2j*Z*t)
    
    #difference = (A1*A2) - B
    difference = (A1 + A2) - B
    # get the maximum eigen value of the difference matrix
    eigenvalues,_ = np.linalg.eig(difference)
    max_eigenvalue = eigenvalues.max()
    other_A.append(max_eigenvalue)
    #print(difference)

## Graph t vs $\epsilon$

In [ ]:
# graph the changes in epsilon respect to the time
plt.plot(time, epsilon, label = "Epsilon")
#plt.plot(time, error_matrix, label = "Remainder function")
#plt.plot(time, other_A, label = "epsilon using sine/cos")

plt.xlabel('Time')
plt.ylabel('$\epsilon$')
plt.legend()

plt.savefig('my_graph.png')
plt.show()

print(time)
print(epsilon)
